In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Python的Vertex AI SDK：AutoML视频分类示例

<table align="left">
<td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/sdk/SDK_AutoML_Video_Classification.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> 在Colab中打开
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fgithub.com%2FGoogleCloudPlatform%2Fvertex-ai-samples%2Fblob%2Fmain%2Fnotebooks%2Fofficial%2Fsdk%2FSDK_AutoML_Video_Classification.ipynb">
      <img width="32px" src="https://cloud.google.com/ml-engine/images/colab-enterprise-logo-32px.png" alt="Google Cloud Colab Enterprise logo"><br> 在Colab Enterprise中打开
    </a>
  </td> 
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/sdk/SDK_AutoML_Video_Classification.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> 在Workbench中打开
    </a>
  </td> 
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/sdk/SDK_AutoML_Video_Classification.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> 在GitHub上查看
    </a>
  </td>                                                                                              
</table>

## 概述

本笔记本演示了如何使用Vertex AI视频数据集创建自动ML视频分类模型，以及如何为批量预测提供模型服务。要求您提供存储数据集的存储桶。

注意：在测试此SDK时，您可能需要为训练、预测、存储或使用其他GCP产品产生费用。

了解更多关于[视频数据的分类](https://cloud.google.com/vertex-ai/docs/training-overview#classification_for_videos)。

### 目标

本笔记本的目标是构建一个自动化机器学习视频分类模型。已遵循以下步骤：  
此教程使用以下谷歌云ML服务：
- Vertex AI 数据集资源
- AutoML 训练
- Vertex AI 模型资源
- Vertex AI 批处理预测

执行的步骤包括：

- 设置您的任务名称和GCS前缀
- 复制AutoML视频演示训练数据以创建托管数据集
- 在Vertex AI上创建数据集。
- 配置训练作业
- 启动训练作业并在Vertex AI上创建模型
- 复制AutoML视频演示预测数据以创建批处理预测作业
- 对模型执行批处理预测作业

### 数据集

##### HMDB: 一个庞大的人类动作数据库
为了演示而准备的一些训练数据和预测数据使用了[HMDB 数据集](https://serre-lab.clps.brown.edu/resource/hmdb-a-large-human-motion-database)。

HMDB 数据集采用知识共享署名4.0国际许可。要查看此许可证的副本，请访问 https://creativecommons.org/licenses/by/4.0/

有关此数据集的更多信息，请访问：https://serre-lab.clps.brown.edu/resource/hmdb-a-large-human-motion-database/

成本

本教程使用Google Cloud的以下计费组件：

- Vertex AI
- Cloud Storage

了解[Vertex AI
定价](https://cloud.google.com/vertex-ai/pricing)，[Cloud Storage
定价](https://cloud.google.com/storage/pricing)，并使用[Pricing
Calculator](https://cloud.google.com/products/calculator/)
根据您的预期使用量生成成本估算。

开始吧

### 为Python安装Vertex AI SDK和其他必需的软件包

In [ ]:
! pip3 install --upgrade --quiet google-cloud-aiplatform

### 重新启动运行时（仅限Colab）

要使用新安装的软件包，您必须在Google Colab上重新启动运行时。

In [ ]:
import sys

if "google.colab" in sys.modules:

    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

<div class="alert alert-block alert-warning">
<b>⚠️ 内核即将重新启动。在继续下一步之前，请等待它完成。 ⚠️</b>
</div>

### 在 Colab 上验证您的笔记本环境

在 Google Colab 上验证您的环境。

In [ ]:
import sys

if "google.colab" in sys.modules:

    from google.colab import auth

    auth.authenticate_user()

### 设置Google Cloud项目信息并为Python初始化Vertex AI SDK

要开始使用Vertex AI，您必须拥有一个现有的Google Cloud项目并[启用Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com)。了解更多关于[设置项目和开发环境](https://cloud.google.com/vertex-ai/docs/start/cloud-environment)的信息。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

# Set the project id
! gcloud config set project {PROJECT_ID}

LOCATION = "us-central1"  # @param {type: "string"}

UUID

为了避免在创建资源时用户之间的名称冲突，请为每个会话实例创建一个uuid。将这些uuid追加到本教程中创建的资源的相应名称上。

In [ ]:
import random
import string


# Generate a uuid of a specifed length(default=8)
def generate_uuid(length: int = 8) -> str:
    return "".join(random.choices(string.ascii_lowercase + string.digits, k=length))


UUID = generate_uuid()

创建一个云存储桶

创建一个存储桶来存储中间产物，如数据集。

使用Cloud SDK提交培训作业时，您需要将包含培训代码的Python软件包上传到云存储桶中。Vertex AI会从该软件包运行代码。在本教程中，Vertex AI还会将作业结果生成的训练模型保存在同一个存储桶中。使用这个模型产物，您可以创建Vertex AI模型资源并用于预测。

In [ ]:
BUCKET_URI = (
    f"gs://your-bucket-name-{PROJECT_ID}-unique-{UUID}"  # @param {type:"string"}
)

只有当您的存储桶尚不存在时：运行以下单元格以创建您的云存储存储桶。

In [ ]:
! gsutil mb -l $LOCATION -p $PROJECT_ID $BUCKET_URI

### 导入库并定义常量

In [ ]:
import json

from google.cloud import aiplatform, storage

MY_PROJECT = PROJECT_ID
MY_STAGING_BUCKET = BUCKET_URI

### 设置您的任务名称和GCS前缀

如果您想要将所有输入和输出文件集中存储在GCS位置下。

In [ ]:
TASK_TYPE = "mbsdk_automl-video-training"
PREDICTION_TYPE = "classification"
MODEL_TYPE = "CLOUD"

TASK_NAME = f"{TASK_TYPE}_{PREDICTION_TYPE}"
BUCKET_NAME = MY_STAGING_BUCKET.split("gs://")[1]
GCS_PREFIX = TASK_NAME

print(f"Bucket Name:    {BUCKET_NAME}")
print(f"Task Name:      {TASK_NAME}")

### 复制 AutoML 视频演示训练数据以创建托管数据集

In [ ]:
automl_video_demo_train_data = (
    "gs://automl-video-demo-data/hmdb_split1_5classes_all.csv"
)


gcs_source_train = f"gs://{BUCKET_NAME}/{TASK_NAME}/data/video_classification.csv"

!gsutil cp $automl_video_demo_train_data $gcs_source_train

使用Vertex AI视频数据集运行AutoML视频训练。

为Python初始化Vertex AI SDK

为Vertex AI初始化*client*。

In [ ]:
aiplatform.init(project=PROJECT_ID, staging_bucket=BUCKET_URI)

在Vertex AI的数据集资源上创建一个数据集
您现在可以使用之前准备的csv文件来创建一个Vertex AI视频数据集。

In [ ]:
dataset = aiplatform.VideoDataset.create(
    display_name=f"temp-{TASK_NAME}",
    gcs_source=gcs_source_train,
    import_schema_uri=aiplatform.schema.dataset.ioformat.video.classification,
    sync=False,
)

In [ ]:
dataset.wait()

启动一个培训工作，并在Vertex AI上创建一个模型。

训练一个AutoML模型，您需要执行两个步骤：1）创建一个训练管道，和2）运行该管道。

### 配置一个训练工作

使用`AutoMLVideoTrainingJob`类创建AutoML训练流水线，具有以下参数：

- `display_name`：`TrainingJob`资源的人类可读名称。
- `prediction_type`：为模型训练的任务类型。
  - `classification`：视频分类模型。
  - `object_tracking`：视频目标跟踪模型。
  - `action_recognition`：视频动作识别模型。

In [ ]:
job = aiplatform.AutoMLVideoTrainingJob(
    display_name=f"temp-{TASK_NAME}",
    prediction_type=PREDICTION_TYPE,
    model_type=MODEL_TYPE,
)

运行训练任务

接下来，通过调用方法`run`来运行作业以启动训练作业，参数如下：

- `dataset`：用于训练模型的`Dataset`资源。
- `model_display_name`：训练模型的可读名称。
- `training_fraction_split`：用于训练的数据集百分比。
- `test_fraction_split`：用于测试（留出数据）的数据集百分比。
- `sync`：如果设置为True，则调用在等待异步批处理作业完成时会阻塞。

完成时，`run`方法将返回模型资源。

训练管道的执行可能需要超过24小时才能完成。

In [ ]:
import os
import sys

if os.getenv("IS_TESTING"):
    sys.exit(0)

In [ ]:
model = job.run(
    dataset=dataset,
    training_fraction_split=0.8,
    test_fraction_split=0.2,
    model_display_name=f"temp-{TASK_NAME}",
    sync=False,
)

In [ ]:
model.wait()

发出批量预测请求

复制AutoML视频演示的预测数据，以创建批量预测作业。

In [ ]:
automl_video_demo_batch_prediction_data = (
    "gs://automl-video-demo-data/hmdb_split1_predict.jsonl"
)

gcs_source_batch_prediction = (
    f"gs://{BUCKET_NAME}/{TASK_NAME}/data/video_classification_batch_prediction.jsonl"
)
gcs_destination_prefix_batch_prediction = (
    f"gs://{BUCKET_NAME}/{TASK_NAME}/batch_prediction"
)

!gsutil cp $automl_video_demo_batch_prediction_data $gcs_source_batch_prediction

### 在模型上执行批量预测任务

现在你的模型资源已经训练完成，你可以通过调用batch_predict()方法进行批量预测，使用以下参数：

- `job_display_name`：批量预测作业的可读名称。
- `gcs_source`：一个或多个批量请求输入文件的列表。
- `gcs_destination_prefix`：用于存储批量预测结果的Cloud Storage位置。
- `sync`：如果设置为True，调用将会阻塞，等待异步批处理作业完成。

In [ ]:
batch_predict_job = model.batch_predict(
    job_display_name=f"temp-{TASK_NAME}",
    gcs_source=gcs_source_batch_prediction,
    gcs_destination_prefix=gcs_destination_prefix_batch_prediction,
    sync=False,
)

In [ ]:
batch_predict_job.wait()

### 获取预测结果

接下来，从已完成的批量预测作业中获取结果。

结果将被写入您在批量预测请求中指定的云存储输出桶中。您调用iter_outputs()方法以获取生成结果的每个云存储文件的列表。每个文件以JSON格式包含一个或多个预测请求：

- `content`：预测请求。
- `prediction`：预测响应。

预测响应包含以下字段：

- `ids`：每个预测请求的内部分配的唯一标识符。
- `displayNames`：每个类别标签的类名。
- `confidences`：每个类别标签的预测置信度，介于0和1之间。
- `timeSegmentStart`：视频中开始视频序列的时间偏移。
- `timeSegmentEnd`：视频中结束视频序列的时间偏移。

In [ ]:
bp_iter_outputs = batch_predict_job.iter_outputs()

prediction_results = list()
for blob in bp_iter_outputs:
    if blob.name.split("/")[-1].startswith("prediction"):
        prediction_results.append(blob.name)
client = storage.Client()
bucket = client.get_bucket(BUCKET_URI.replace("gs://", ""))
for prediction_result in prediction_results:
    gfile_name = f"{prediction_result}"
    data = bucket.blob(gfile_name).download_as_string()
    data = json.loads(data)
    print(data)

## 清理
<a name="section-13"></a>

要清理本项目中使用的所有Google Cloud资源，您可以[删除用于本教程的Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除本教程中创建的各个资源。

In [ ]:
delete_bucket = False

# Delete the dataset using the Vertex dataset object
dataset.delete()

# Delete the model using the Vertex model object
model.delete()

# Delete the AutoML or Pipeline training job
job.delete()

# Delete the batch prediction job using the Vertex batch prediction object
batch_predict_job.delete()

# Delete the Cloud Storage bucket
if delete_bucket:
    ! gsutil -m rm -r $BUCKET_URI